# Regression and Other Stories: Pew

Miscellaneous analyses using raw Pew data. See Chapter 2 in Regression and Other Stories.

-----

### Load packages

In [18]:
using DataFrames, StatsPlots, CSV, HTTP, StatsBase
using StatFiles # For .dta file (stata)

### Load data

In [3]:
# Not aware of a way to pass url to read_dta, so downloaded data locally
pew_pre = load("data/pew_research_center_june_elect_wknd_data.dta") |> DataFrame
n = nrow(pew_pre);

31201

In [59]:
first(pew_pre, 6)

,survey,rid,date,sample,phoneuse,zipcode,msa,usr,form,thoughtpres
,Int32?,Int32?,Int32?,Int8?,Int8?,String?,Int32?,String?,Int8?,Int8?
1,20080629,1720,62708,1,2,01007,44140,S,2,missing
2,20080810,668,80208,1,2,01010,44140,2,1,missing
3,20080810,50,73108,1,3,01013,44140,2,1,missing
4,20080810,50533,80208,2,3,01013,44140,,2,missing
5,20080629,30091,62108,3,4,01020,44140,S,1,missing
6,20080727,60,72708,1,0,01020,44140,S,2,missing


### Glance data

In [24]:
# A little less pretty than R's `table` method
println(sort(countmap(pew_pre.date), by=x->x[1]));

OrderedCollections.OrderedDict{Union{Missing, Int32}, Int64}(61808 => 228, 61908 => 422, 62008 => 340, 62108 => 412, 62208 => 376, 62308 => 442, 62408 => 420, 62508 => 280, 62608 => 370, 62708 => 258, 62808 => 334, 62908 => 126, 72308 => 484, 72408 => 942, 72508 => 502, 72608 => 654, 72708 => 424, 73108 => 360, 80108 => 584, 80208 => 508, 80308 => 478, 80408 => 524, 80508 => 742, 80608 => 410, 80708 => 532, 80808 => 560, 80908 => 540, 81008 => 540, 81108 => 32, 90908 => 614, 91008 => 1420, 91108 => 1184, 91208 => 976, 91308 => 970, 91408 => 800, 92708 => 1498, 92808 => 1246, 92908 => 266, 100908 => 376, 101008 => 384, 101108 => 433, 101208 => 285, 101308 => 7, 101608 => 874, 101708 => 922, 101808 => 843, 101908 => 377, 102308 => 535, 102408 => 527, 102508 => 400, 102608 => 38, 102908 => 873, 103008 => 867, 103108 => 864, 110108 => 798)


In [33]:
which_question = ifelse.(
    .!ismissing.(pew_pre.heat2),
    2,
    ifelse.(
        .!ismissing(pew_pre.heat4),
        4,
        0
    )
);

In [41]:
#sort(pew_pre.date)
# TODO

### Create vote intention variable "rvote" using variables heat2 and heat4 from Pew

In [49]:
numeric_heat2 = replace(pew_pre.heat2, missing=>NaN)
numeric_heat4 = replace(pew_pre.heat4, missing=>NaN)
rvote = fill(NaN, n)
for i in 1:n
    if(which_question[i]==2)
        rvote[i] = ifelse.(numeric_heat2[i]==1,1,
                    ifelse.(numeric_heat2[i]==2,0,NaN))
    elseif(which_question[i]==4)
        rvote[i] = ifelse.(numeric_heat4[i]==1,1,
                    ifelse.(numeric_heat4[i]==2,0,NaN))
    end
end

### Certain to have registered to vote?

In [69]:
countmap(pew_pre[!,:regicert])
# This is different than represented in the original notebook...

Dict{Union{Missing, Int8}, Int64} with 4 entries:
  2       => 1078
  9       => 114
  missing => 3820
  1       => 26189